In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install datatable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 9.0 MB/s eta 0:00:00


In [13]:
pip install lightgbm==3.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.1.0
    Uninstalling lightgbm-4.1.0:
      Successfully uninstalled lightgbm-4.1.0


In [14]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("/content/drive/MyDrive/millennium_project")

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output
import lightgbm as lgb

# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

[autoreload of lightgbm.callback failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: __repr__() requires a code object with 2 free vars

In [5]:
ticker_name = "GOOG"

In [6]:
N_FOLD = 5
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = ticker_name+'-lgb'
DATA_NAME = ticker_name
mkdir(f'./drive/MyDrive/millennium_project/models/{SOL_NAME}/')

# Functions

In [7]:
def feval_rmse(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSE', rmse(y_true, y_pred), False

# Loading data

In [8]:
# train
df_train = dt.fread(f'./drive/MyDrive/millennium_project/dataset/train_{DATA_NAME}_LGB.csv').to_pandas()
fea_cols = fea_cols_GOOG
df_train["name"] = ticker_name
# result
df_result = dt.fread(f'./drive/MyDrive/millennium_project/dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [9]:
df_train = gen_row_id(df_train)
df_train = add_time_fold(df_train, N_FOLD)

In [10]:
df_train.index = df_train.row_id
df_result.index = df_result.row_id

# Evaluation

In [15]:
seed0=2021
params = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':100,
    'min_data_in_leaf':500,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.5,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
list_rmse = [1 for _ in range(N_FOLD)]

In [17]:
n_trials = 1
for _ in range(n_trials):
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]

        X_train = df_tr[fea_cols].values
        y_train = df_tr['value_to_predict'].values
        X_test = df_te[fea_cols].values
        y_test = df_te['value_to_predict'].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)


        train_dataset = lgb.Dataset(X_train, y_train, weight=1/np.square(y_train))
        test_dataset = lgb.Dataset(X_test, y_test, weight=1/np.square(y_test))
        model = lgb.train(params = params,
                          num_boost_round=1000,
                          train_set = train_dataset,
                          valid_sets = [train_dataset, test_dataset],
                          verbose_eval = 250,
                          early_stopping_rounds=250,
                          feval = feval_rmse)

        y_pred = model.predict(X_test)
        curr_rmse = rmse(y_test, y_pred)
        if curr_rmse < list_rmse[i_fold]:
            ckp_path = f'./models/{SOL_NAME}/lgb_{i_fold}.pkl'
            #save_pickle(model, ckp_path)
            list_rmse[i_fold] = curr_rmse
            # generate and save preds
            df_result.loc[idx_test, 'pred'] = y_pred
        # clear_output()
        print(list_rmse)
        calc_metric(df_result.fillna(0))
        # 0.2169
    # break

Fold 1/5 (3730, 15) (933, 15)
Training until validation scores don't improve for 250 rounds


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[250]	training's rmse: 0.0389978	training's RMSE: 0.69304	valid_1's rmse: 0.0106943	valid_1's RMSE: 0.693251
Early stopping, best iteration is:
[2]	training's rmse: 0.00644051	training's RMSE: 0.712084	valid_1's rmse: 0.00639728	valid_1's RMSE: 0.7122
[0.712200332971195, 1, 1, 1, 1]
   R2: -1.7042
RMSE: 0.6672
Fold 2/5 (3730, 15) (933, 15)
Training until validation scores don't improve for 250 rounds


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[250]	training's rmse: 0.0121795	training's RMSE: 0.703917	valid_1's rmse: 0.0192165	valid_1's RMSE: 0.704207
Early stopping, best iteration is:
[10]	training's rmse: 0.0121772	training's RMSE: 0.713246	valid_1's rmse: 0.00291766	valid_1's RMSE: 0.71365
[0.712200332971195, 0.7136498617896687, 1, 1, 1]
   R2: -1.7041
RMSE: 0.6672
Fold 3/5 (3730, 15) (933, 15)
Training until validation scores don't improve for 250 rounds


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[250]	training's rmse: 0.00533603	training's RMSE: 0.708931	valid_1's rmse: 0.0383253	valid_1's RMSE: 0.708836
[500]	training's rmse: 0.00946631	training's RMSE: 0.704857	valid_1's rmse: 0.0379543	valid_1's RMSE: 0.704777
Early stopping, best iteration is:
[294]	training's rmse: 0.0121205	training's RMSE: 0.704128	valid_1's rmse: 0.0379804	valid_1's RMSE: 0.704
[0.712200332971195, 0.7136498617896687, 0.7072093027949653, 1, 1]
   R2: -1.7022
RMSE: 0.6670
Fold 4/5 (3731, 15) (932, 15)
Training until validation scores don't improve for 250 rounds


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[250]	training's rmse: 0.00547729	training's RMSE: 0.709567	valid_1's rmse: 0.0819769	valid_1's RMSE: 0.709422
Early stopping, best iteration is:
[182]	training's rmse: 0.00972122	training's RMSE: 0.706718	valid_1's rmse: 0.0817313	valid_1's RMSE: 0.706566
[0.712200332971195, 0.7136498617896687, 0.7072093027949653, 0.7095253469431584, 1]
   R2: -1.7010
RMSE: 0.6668
Fold 5/5 (3731, 15) (932, 15)
Training until validation scores don't improve for 250 rounds
[250]	training's rmse: 0.0154492	training's RMSE: 0.703017	valid_1's rmse: 0.0293044	valid_1's RMSE: 0.702717
Early stopping, best iteration is:
[12]	training's rmse: 0.00536973	training's RMSE: 0.71294	valid_1's rmse: 0.0233825	valid_1's RMSE: 0.712662
[0.712200332971195, 0.7136498617896687, 0.7072093027949653, 0.7095253469431584, 0.7117321257710167]
   R2: -1.7005
RMSE: 0.6667


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [18]:
df_result.to_csv(f'./drive/MyDrive/millennium_project/results/{SOL_NAME}.csv', index=False)

In [19]:
list_rmse

[0.712200332971195,
 0.7136498617896687,
 0.7072093027949653,
 0.7095253469431584,
 0.7117321257710167]